## Part 9 - Random Forest

In [3]:
import pip 
pip.main(["install","matplotlib"]) 

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


0

In [7]:
!pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in c:\python39\lib\site-packages (1.1.2)


You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [30]:
# Chargement des librairies nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_curve
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) # to avoid deprecation warnings

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [12]:
# Chargement de la dataset
dataset = pd.read_csv("dataset_CV_labelise.csv", delimiter = ";", encoding = "ISO-8859-1")
dataset.head()

,CV_Sentences,Sentences_CV_clean,CV_Number,Sentence_line,Nb_tokens,%texte_lu,%texte_lu_fin_ligne,Is_alpha,Grammar,Label
0,SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE,SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE,CV_1,0.0,6.0,2.80,97.20,"[True, True, True, False, True, True]","['NOUN', 'PROPN', 'VERB', 'NUM', 'PROPN', 'PRO...",1
1,PROFIL,PROFIL,CV_1,1.0,1.0,3.27,96.73,[True],['NOUN'],0
2,PERSONNEL Je suis Ã©tudiante au lycÃ©e Condorcet.,PERSONNEL Je suis Ã©tudiante au lycÃ©e Condorcet.,CV_1,2.0,8.0,7.01,92.99,"[True, True, True, True, True, True, True, False]","['NOUN', 'PRON', 'AUX', 'ADJ', 'ADP', 'NOUN', ...",0
3,Je code depuis l'Ã¢ge de 13 ans et j'aime crÃ©...,Je code depuis l'Ã¢ge de 13 ans et j'aime crÃ©...,CV_1,3.0,24.0,18.22,81.78,"[True, True, True, False, True, True, False, T...","['PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', ...",0
4,RÃALISATIONS,RÃALISATIONS,CV_1,4.0,1.0,18.69,81.31,[True],['NOUN'],0


In [13]:
dataset.shape

(4049, 10)

In [14]:
data_desc = dataset.describe(include='all')
display(data_desc)

,CV_Sentences,Sentences_CV_clean,CV_Number,Sentence_line,Nb_tokens,%texte_lu,%texte_lu_fin_ligne,Is_alpha,Grammar,Label
count,4049,4045,4047,4047.000000,4047.000000,4047.000000,4047.000000,4047,4047,4049.000000
unique,2829,2825,145,NaN,NaN,NaN,NaN,1251,2058,NaN
top,#NOM?,#NOM?,CV_10,NaN,NaN,NaN,NaN,[False],['NOUN'],NaN
freq,131,131,66,NaN,NaN,NaN,NaN,463,249,NaN
mean,NaN,NaN,NaN,15.822090,9.097850,50.400032,49.599970,NaN,NaN,0.040257
std,NaN,NaN,NaN,11.372538,10.625067,30.183407,30.183408,NaN,NaN,0.196585
min,NaN,NaN,NaN,0.000000,1.000000,0.240000,0.000000,NaN,NaN,0.000000
25%,NaN,NaN,NaN,7.000000,2.000000,24.310000,23.635000,NaN,NaN,0.000000
50%,NaN,NaN,NaN,14.000000,5.000000,49.760000,50.240000,NaN,NaN,0.000000
75%,NaN,NaN,NaN,23.000000,12.000000,76.365000,75.690000,NaN,NaN,0.000000


In [15]:
display(100*dataset.isnull().sum()/dataset.shape[0])

CV_Sentences           0.000000
Sentences_CV_clean     0.098790
CV_Number              0.049395
Sentence_line          0.049395
Nb_tokens              0.049395
%texte_lu              0.049395
%texte_lu_fin_ligne    0.049395
Is_alpha               0.049395
Grammar                0.049395
Label                  0.000000
dtype: float64

In [16]:
dataset["Label"].unique()
dataset = dataset.loc[(dataset['CV_Sentences'] != "#NOM?"),:]
dataset.shape 

(3918, 10)

In [17]:
dataset = dataset.loc[(dataset['CV_Sentences'] != ":"),:]
dataset.shape

(3914, 10)

In [18]:
dataset = dataset.dropna(axis =0, how = 'any')
dataset.shape

(3910, 10)

In [19]:
display(100*dataset.isnull().sum()/dataset.shape[0])

CV_Sentences           0.0
Sentences_CV_clean     0.0
CV_Number              0.0
Sentence_line          0.0
Nb_tokens              0.0
%texte_lu              0.0
%texte_lu_fin_ligne    0.0
Is_alpha               0.0
Grammar                0.0
Label                  0.0
dtype: float64

In [21]:
print("Separating labels from features...")
col_list = ['CV_Sentences',"Sentences_CV_clean","CV_Number", "Is_alpha", "Grammar", "Label"]
target_variable = "Label"

X = dataset.drop(col_list, axis = 1)
y = dataset.loc[:,target_variable]

print('y : ')
print(y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
y : 
0    1
1    0
2    0
3    0
4    0
Name: Label, dtype: int64

X :
   Sentence_line  Nb_tokens  %texte_lu  %texte_lu_fin_ligne
0            0.0        6.0       2.80                97.20
1            1.0        1.0       3.27                96.73
2            2.0        8.0       7.01                92.99
3            3.0       24.0      18.22                81.78
4            4.0        1.0      18.69                81.31


In [22]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i,t in X.dtypes.iteritems():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
    else :
        categorical_features.append(i)

print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['Sentence_line', 'Nb_tokens', '%texte_lu', '%texte_lu_fin_ligne']
Found categorical features  []


In [23]:
# Divide dataset Train set & Test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

In [24]:
# Create pipeline for numeric features
numeric_transformer = SimpleImputer(strategy='mean')

In [25]:
# Create pipeline for categorical features
categorical_transformer = OneHotEncoder(drop='first')

In [26]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [32]:
# Perform grid search
print("Grid search...")
classifier = RandomForestClassifier()

# Grid of values to be tested
params = {
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 5],
    'min_samples_split': [2, 4, 8],
    'n_estimators': [10, 20, 40, 60, 80, 100]
}
gridsearch = GridSearchCV(classifier, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best validation accuracy : ", gridsearch.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'max_depth': 8, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 80}
Best validation accuracy :  0.9731454678510548


In [33]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = gridsearch.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()

Predictions on training set...
...Done.
[0 0 0 ... 0 0 0]



In [34]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = gridsearch.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()

Predictions on test set...
...Done.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0

In [38]:
# Print scores
print("accuracy on training set : ", accuracy_score(y_train, Y_train_pred))
print("accuracy on test set : ", accuracy_score(y_test, Y_test_pred))
print()

print("f1-score on training set : ", f1_score(y_train, Y_train_pred))
print("f1-score on test set : ", f1_score(y_test, Y_test_pred))                              
print()

accuracy on training set :  0.9785805626598465
accuracy on test set :  0.9744245524296675

f1-score on training set :  0.6940639269406392
f1-score on test set :  0.6296296296296297



In [40]:
# Visualize confusion matrices
from plotly.subplots import make_subplots
cm_train = confusion_matrix(y_train, Y_train_pred)
cm_test = confusion_matrix(y_test, Y_test_pred)

fig = make_subplots(rows = 1, cols = 2, subplot_titles = ("train", "test"), 
                    x_title = 'Prediction', y_title = 'True label')
fig.update_layout(
        title = go.layout.Title(text = "Confusion matrices", x = 0.5))
fig.update_yaxes(autorange='reversed')
fig.add_trace(
    go.Heatmap(
        name = 'train',
        x = ['0', '1'], 
        y = ['0', '1'], 
        z = cm_train,
        colorscale = 'gnbu',
        zmin = 0,
        zmax = max(cm_train.max(), cm_test.max())
    ),
    row = 1,
    col = 1
)  
fig.add_trace(
    go.Heatmap(
        name = 'test',
        x = ['0', '1'], 
        y = ['0', '1'], 
        z = cm_test,
        colorscale = 'gnbu',
        zmin = 0,
        zmax = max(cm_train.max(), cm_test.max())
    ),
    row = 1,
    col = 2
)

In [43]:
# Visualize ROC curves
probas_train = gridsearch.predict_proba(X_train)[:,1]
precisions, recalls, thresholds = roc_curve(y_train, probas_train)
fig = go.Figure(
    data = go.Scatter(
        name = 'train',
        x = recalls, 
        y = precisions, 
        mode = 'lines'
    ),
    layout = go.Layout(
        title = go.layout.Title(text = "ROC curve", x = 0.5),
        xaxis = go.layout.XAxis(title = 'False Positive Rate'),
        yaxis = go.layout.YAxis(title = 'True Positive Rate')
    )
)

probas_test = gridsearch.predict_proba(X_test)[:,1]
precisions, recalls, thresholds = roc_curve(y_test, probas_test)
fig.add_trace(go.Scatter(
    name = 'test',
    x = recalls, 
    y = precisions, 
    mode = 'lines'
    )
)
fig.show()